In [11]:
import openai
import pandas as pd

# Initialize OpenAI API key
openai.api_key = ""

# Load and preprocess laptop data
laptop_df = pd.read_csv("/content/laptop_data.csv")
laptop_df['Price'] = laptop_df['Price'].str.replace(',', '').astype(int)

# Step 1: Feature Extraction using Function Calling
def extract_laptop_features(laptop_description):
    lap_spec = {
        "GPU intensity": "(Type of the Graphics Processor)",
        "Display quality": "(Display Type, Screen Resolution, Display Size)",
        "Portability": "(Laptop Weight)",
        "Multitasking": "(RAM Size)",
        "Processing speed": "(CPU Type, Core, Clock Speed)"
    }

    prompt = f"""
    Extract and classify laptop features from the following description according to:
    GPU intensity, display quality, portability, multitasking, and processing speed.
    Classify each feature as 'low', 'medium', or 'high' based on the description.

    Laptop Description: {laptop_description}
    Specifications: {lap_spec}
    """

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a laptop feature extractor."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# Step 2: Recommendation Processing Based on User Requirements
def recommend_laptops(user_requirements):
    budget = int(user_requirements.get('Budget', '0').replace(',', '').split()[0])

    # Filter laptops within budget
    filtered_laptops = laptop_df[laptop_df['Price'] <= budget]

    prompt = f"""
    Rank laptops based on the user's requirements within the given budget:
    Requirements: {user_requirements}
    Laptops: {filtered_laptops.to_dict(orient='records')}
    """

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a laptop recommendation engine."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# Step 3: Dynamic Conversation Flow
def conversation_flow(user_input, user_profile=None):
    conversation_context = {
        "profile": user_profile or {},
        "user_input": user_input,
        "product_data": "laptop_data.csv"
    }

    prompt = f"""
    Given the user profile and input, provide a natural, informative response that suggests suitable laptops.
    Ask clarifying questions or provide recommendations as needed.
    Conversation Context: {conversation_context}
    """

    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a conversational laptop recommendation assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# Sample Workflow to Test the Integrated Code

# Use laptop descriptions from laptop_df for feature extraction
for index, row in laptop_df.iterrows():
    laptop_description = row['Description']  # Assumes 'Description' column in laptop_df

    # Step 1: Extract and classify laptop features
    features = extract_laptop_features(laptop_description)
    print(f"Laptop {index + 1} Extracted Features:", features)

# Sample user requirements for recommendation
user_requirements = {
    "Budget": "40000",
    "GPU intensity": "medium",
    "Portability": "medium",
    "Multitasking": "medium"
}

# Step 2: Get recommendations based on user requirements
recommendations = recommend_laptops(user_requirements)
print("Recommendations:", recommendations)

# Step 3: Simulate refined conversation
user_input = "Which laptop is ideal for travel?"
user_profile = {
    "GPU intensity": "medium",
    "Portability": "high"
}
conversation_response = conversation_flow(user_input, user_profile)
print("Conversation Response:", conversation_response)


Laptop 1 Extracted Features: GPU intensity: Medium (Intel UHD GPU)
Display quality: High (15.6" LCD display, 1920x1080 resolution)
Portability: High (2.5 kg laptop weight)
Multitasking: Medium (8GB RAM)
Processing speed: Medium (Intel Core i5 processor, 2.4 GHz)
Laptop 2 Extracted Features: GPU intensity: High (Equipped with an NVIDIA GTX graphics card)
Display quality: High (15.6" IPS display with a resolution of 1920x1080)
Portability: Medium (Weighing just 2.3 kg)
Multitasking: High (16GB of RAM)
Processing speed: High (Intel Core i7 processor running at 2.6 GHz)
Laptop 3 Extracted Features: GPU intensity: Medium (Intel UHD GPU)
Display quality: High ('14" LED display', 'Resolution of 1920x1080')
Portability: High ('Weight of 1.5kg')
Multitasking: High ('16GB of RAM')
Processing speed: High ('Intel Core i7 processor', '2.8 GHz')
Laptop 4 Extracted Features: GPU intensity: Low (Intel UHD GPU)
Display quality: Medium (15.6" TN display, Resolution of 1366x768)
Portability: Medium (Weig